In [1]:
import pandas as pd


In [2]:

distances = pd.read_csv('dist.csv')
depos = pd.read_csv('depos.csv')
repairs_costs = pd.read_csv('repairs_costs.csv')
stations = pd.read_csv('stations.csv')
vagons_repairs = pd.read_csv('vagons_repairs.csv')
vagons_ways = pd.read_csv('vagons_ways.csv')

all_data = []

In [11]:
distances.head(30)

,station_id,name,depo_id,depo_name,distance_to_depo
0,1,Сосновка,1,Депо-Ленинское,10077
1,1,Сосновка,2,Депо-Комсомольское,10436
2,1,Сосновка,3,Депо-Молодежное,10522
3,2,Липовка,1,Депо-Ленинское,2084
4,2,Липовка,2,Депо-Комсомольское,3789
5,2,Липовка,3,Депо-Молодежное,7637
6,3,Рожок,1,Депо-Ленинское,9936
7,3,Рожок,2,Депо-Комсомольское,2341
8,3,Рожок,3,Депо-Молодежное,4198
9,4,Гатка,1,Депо-Ленинское,7157


In [4]:
depos.head()

,id,depo_name
0,1,Депо-Ленинское
1,2,Депо-Комсомольское
2,3,Депо-Молодежное


In [8]:
vagons_repairs.head()

,car_num,ct_name,psevdoname,std_kap,std_ppr,std_dep,std_vogi
0,57463085,Газовые,15-78-5,2021-08-08,2018-08-26,2019-08-31,2019-05-19
1,57456220,Газовые,15-78-5,2021-08-02,2019-05-19,2019-09-15,2019-05-19
2,57453656,Газовые,15-78-5,2021-07-19,2019-01-22,2019-07-07,2019-05-18
3,57463507,Газовые,15-78-5,2021-07-19,2019-03-18,2018-06-02,2019-05-18
4,57463309,Газовые,15-78-5,2021-06-20,2018-05-16,2019-01-09,2019-04-24


In [35]:
len(vagons_ways['car_num'].unique())

17988

In [7]:
vagons_repairs[vagons_repairs['car_num'] == 57751372]

,car_num,ct_name,psevdoname,std_kap,std_ppr,std_dep,std_vogi
194,57751372,Газовые,11-160-9,2020-03-26,2018-03-17,2018-05-25,2018-03-17


In [39]:
# vagons_ways.head()


,car_num,date,station_id
0,57463085,2015-01-01,2
1,57463085,2015-01-25,6
2,57463085,2015-02-18,9
3,57463085,2015-03-14,5
4,57463085,2015-04-07,8
5,57463085,2015-05-01,2
6,57463085,2015-05-25,6
7,57463085,2015-06-18,9
8,57463085,2015-07-12,3
9,57463085,2015-08-05,6
